In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import glob
import imageio
import datetime
%load_ext tensorboard
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
def processData(imPath, labelPath):
    df1 = pd.read_csv(labelPath, header=0)
    yEncoder = OneHotEncoder(handle_unknown='ignore')
    yEncoder.fit(df1[['label']])
    #labelNames = yEncoder.get_feature_names()
    labels = yEncoder.transform(df1[['label']]).toarray()
    df2 = pd.DataFrame(data=labels)
    df1.drop(['label'], axis=1, inplace=True)
    df = pd.concat([df1, df2], axis=1)
    record = df.set_index('id').T.to_dict('list')
    #print(record[3])
    data = []
    imPaths = glob.glob(imPath)
    for i in imPaths:
        #print(i.split('.')[1][7:])
        ID = int(i.split('.')[1][7:])
        tmp = imageio.imread(i)
        '''
        r = tmp[:,:,0]
        g = tmp[:,:,1]
        b = tmp[:,:,2]
        rgb=np.array([r, g, b])
        '''
        tmp = [tmp/255.0, record[ID]]
        data.append(tmp)
    return data

In [3]:
#df = pd.read_csv('./trainLabels.csv', header=0)
#df[['label']]
data = processData('./train/*.png', './trainLabels.csv')
#imPaths = glob.glob('./train/*.png')
#imPaths = sorted(imPaths, key=lambda x:x[10:-3])
#print(imPaths)
#images = np.array([])
#images = []
#for i in imPaths:
    #images.append(imageio.imread(i))
    #images = np.append(images, imageio.imread(i))
#print(len(images))
#print(images.shape)

In [4]:
train, test = train_test_split(data, test_size=0.2)
trainX = np.array([i[0] for i in train])
trainY = np.array([i[1] for i in train])
testX = np.array([i[0] for i in test])
testY = np.array([i[1] for i in test])

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', input_shape=(32, 32, 3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size=(4, 4), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        16416     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 32)          0

In [6]:
model.fit(trainX, trainY, batch_size=500, epochs=30, verbose=1)

Epoch 1/30
16/80 [=====>........................] - ETA: 22s - loss: 2.0536 - accuracy: 0.2690

KeyboardInterrupt: 

In [ ]:
model.evaluate(testX,  testY)